# End of week 2 exercise

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports
import os, json
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display
import gradio as gr
from transformers import pipeline
import numpy as np
import base64
from io import BytesIO
from PIL import Image

load_dotenv(override=True)

In [ ]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_GEMINI = 'gemini-3-flash-preview'

openai = OpenAI()
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini=OpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
# set up environment

system_prompt="""
You're an expert in LLM Engineering and particularly in building Agent skills for Claude Code.
You will use this documentation as your guidelines: https://code.claude.com/docs/en/skills
Your main role is to help users to improve there agents and explain how they can improve these agents by themselves.
You will return improved and structured agent skills, then explainations.
Answer only in Markdown.
You can ask question to have more context if needed.
"""

ux_system_prompt="""
You're an expert in UX and UI.
You are able to take a short brief and improve it into a prompt.
You'll optimize the prompt for Dall-E 3 to generate an example of this UX.
Do not comment.
Only return a full text prompt
"""

In [ ]:
def artist(prompt):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
ux_function = {
    "name": "get_ux",
    "description": "Get a valid prompt for a described UI or UX.",
    "parameters": {
        "type": "object",
        "properties": {
            "described_ux": {
                "type": "string",
                "description": "The description of a desired ux",
            },
        },
        "required": ["described_ux"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": ux_function}]

In [ ]:
def get_ux(described_ux):
    print('described_ux : ' + described_ux)

    messages = [
        {"role": "system", "content": ux_system_prompt},
        {"role": "user", "content": described_ux}
    ]
    prompt = gemini.chat.completions.create(
        model=MODEL_GEMINI,
        messages=messages
    )

    print('prompt : ' + prompt.choices[0].message.content)

    return prompt.choices[0].message.content

In [ ]:
def handle_tool_calls(message):
    responses = []
    described_uxs = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ux":
            print('tool called')
            arguments = json.loads(tool_call.function.arguments)
            described_ux = arguments.get('described_ux')
            
            actual_described_ux = get_ux(described_ux)
            described_uxs.append(actual_described_ux)
            responses.append({
                "role": "tool",
                "content": actual_described_ux,
                "tool_call_id": tool_call.id
            })
    return responses, described_uxs

In [ ]:
# Handles chat window
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)
    described_uxs = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, described_uxs = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    if described_uxs:
        image = artist(described_uxs[0])
    
    return history, described_uxs, image

In [ ]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        uxs = gr.Textbox(lines=30)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, uxs, image_output]
    )

ui.launch(inbrowser=True, auth=("blt909", "coconuts"))